In [2]:
print('hello')

hello


In [3]:
pwd

'/gsfs0/data/aglinsai'

In [ ]:
#!ps  -ef | grep $$ | grep -v grep

In [5]:
import os
import numpy as np
#import ants
from matplotlib import pyplot as plt
from nilearn.input_data import NiftiMasker

import datetime
now = datetime.datetime.now

In [6]:
bids_dir= '/gsfs0/data/aglinsai/fMRI_InData/DS-use-new-atlas'

In [7]:
subs = [sub for sub in os.listdir(bids_dir) if 'sub-' in sub]
subs.sort()

In [8]:
#s = 0;mtemp='mv2Mask_3001'
def is_50(s,mtemp):
    sdir=os.path.join(bids_dir,subs[s])
    sadir=os.path.join(sdir,'atlas')
    conts = os.listdir(sadir);conts.sort()
    files = [c for c in conts if mtemp in c and '.nii' in c];files.sort()
    return len(files)==50

#done_subs = np.argwhere([is_50(s,mtemp) for s in range(len(subs))])
#len(done_subs)

In [12]:
#mtemp = 'rfx2Mask_3001'
mtemp='rfx2Mask_3001'
done_subs = np.argwhere([is_50(s,mtemp) for s in range(len(subs))])
len(done_subs)

15

In [ ]:
def get_ch_vec(s,mtemp):
    #s = 1
    sdir=os.path.join(bids_dir,subs[s])
    sadir=os.path.join(sdir,'atlas')
    conts = os.listdir(sadir);conts.sort()

    nifti_masker = NiftiMasker(mask_img=os.path.join(sadir,'epimask.nii'),)

    #mtemp='mv2Mask_3001'
    files = [c for c in conts if mtemp in c and '.nii' in c];files.sort()

    data0 = nifti_masker.fit_transform(os.path.join(sadir,'imask.nii'))
    data1 = nifti_masker.fit_transform(os.path.join(sadir,files[0]))
    for i in range(1,len(files)):
        t = nifti_masker.fit_transform(os.path.join(sadir,files[i]))
        data1 = np.vstack((data1,t))

    ch_vec = np.array(data1==data0).sum(axis=1)/data1.shape[1]

    return ch_vec

In [ ]:
mtemp='mv2Mask_3001'

done_subs = np.argwhere([is_50(s,mtemp) for s in range(len(subs))])
len(done_subs)

print(now())
print('stacking');
ch_vecs=get_ch_vec(int(done_subs[0]),mtemp)
for i in done_subs[1::]:
    ch_vecs = np.vstack(  (ch_vecs,get_ch_vec(int(i),mtemp)) )
    print('{}/{}'.format(ch_vecs.shape[0],len(done_subs[1::])))
print(now())

plt.figure(figsize=(10,7.5))
plt.plot(ch_vecs.transpose());

In [ ]:
plt.hist(ch_vecs[:,0],alpha=.5)
plt.hist(ch_vecs[:,-1],alpha=.5)
plt.legend(['0','50'])

In [ ]:
mtemp='rfx2Mask_3001'

done_subs = np.argwhere([is_50(s,mtemp) for s in range(len(subs))])
len(done_subs)

print(now())
print('stacking');
ch_vecs=get_ch_vec(int(done_subs[0]),mtemp)
for i in done_subs[1::]:
    ch_vecs = np.vstack(  (ch_vecs,get_ch_vec(int(i),mtemp)) )
    print('{}/{}'.format(ch_vecs.shape[0],len(done_subs[1::])))
print(now())

plt.figure(figsize=(10,7.5))
plt.plot(ch_vecs.transpose());

In [ ]:
plt.hist(ch_vecs[:,0],alpha=.5)
plt.hist(ch_vecs[:,-1],alpha=.5)
plt.legend(['0','50'])

In [14]:
#plt.bar(np.arange(ch_vecs.shape[0]),ch_vecs[:,0])
#plt.bar(np.arange(ch_vecs.shape[0]),ch_vecs[:,49])